# <p style="text-align:center"> **GIAI ĐOẠN 4: XÂY DỰNG VÀ ĐÁNH GIÁ MÔ HÌNH** </p>


### <p style="text-align:center"> **I. IMPORT THƯ VIỆN CẦN THIẾT** </p>

In [96]:
# Import thư viện cần thiết để thao tác dữ liệu
import pandas as pd
import numpy as np 

# Import thư viện để trực quan những kết quả tìm hiểu được:
import matplotlib.pyplot as plt

#Import thư viện hỗ trợ xây dựng mô hình
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score,KFold, RepeatedStratifiedKFold, GridSearchCV, RandomizedSearchCV

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

from sklearn.metrics import mean_squared_error

In [97]:
#Lấy các biến từ pha trước vào
%store -r dfMidfielder
%store -r dfKeeper
%store -r dfForward
%store -r dfDefender

In [98]:
#Lưu trữ các biến lại để tránh bị dính các note ở các giá trị chưa được khai báo 
dfMidfieder = dfMidfielder
dfKeeper = dfKeeper
dfForward = dfForward
dfDefender = dfDefender

### <p style="text-align:center"> **II.XỬ LÍ DỮ LIỆU ĐỂ PHÙ HỢP XÂY DỰNG MÔ HÌNH** </p>

Đầu tiên, vì tính chất của bộ dữ liệu không đồng nhất về các cột đặc điểm, ta sẽ không sử dụng dữ liệu của các thủ môn để xây dựng mô hình. Kế tiếp, vì các đặc điểm của tiền vệ, hậu vệ, tiền đạo cũng không đồng nhất; ta chỉ sử dụng những đặc tính chung của loại cầu thủ đó mà thôi.

In [99]:
#Lấy các thuộc tính giống nhau trong 3 bộ dữ liệu của tiền vệ, tiền đạo và hậu vệ
colMidfielder = dfMidfieder.columns
colForward = dfForward.columns
colDefender = dfDefender.columns

same = [i for i in colMidfielder if i in colForward and i in colDefender]
print(same)

['Player name', 'Appearances', 'Wins', 'Losses', 'Headed goals', 'Goals with right foot', 'Goals with left foot', 'Hit woodwork', 'Assists', 'Passes', 'Big chances created', 'Crosses', 'Yellow cards', 'Red cards', 'Fouls', 'Offsides', 'Tackles', 'Blocked shots', 'Interceptions', 'Clearances']


In [100]:
#Loại bỏ các cột dữ liệu dư thừa
dfDefender = dfDefender[dfDefender.columns.intersection(same)]
dfForward = dfForward[dfForward.columns.intersection(same)]
dfMidfieder = dfMidfieder[dfMidfieder.columns.intersection(same)]

In [101]:
#Thêm một cột vị trí ở cuối cùng
dfDefender['Position'] = 'DF'
dfForward['Position'] = 'FW'
dfMidfieder['Position'] = 'MF'

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_20892\2615243928.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfDefender['Position'] = 'DF'
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_20892\2615243928.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfForward['Position'] = 'FW'
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_20892\2615243928.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

In [102]:
#Gộp ba bảng lại thành một dataframe thống nhất
df = pd.concat([dfDefender, dfForward, dfMidfieder], ignore_index=True)

In [103]:
#Kiểm tra kích cỡ của bộ dữ liệu
df.shape

(908, 21)

In [104]:
#Loại bỏ cột tên (không cần thiết cho việc xây dựng mô hình)
df.drop(columns = 'Player name', inplace=True)

Đưa dữ liệu về dạng trung bình mỗi trận đấu để khoảng cách giữa dữ liệu của các cầu thủ mới chơi
và các cầu thủ lâu năm không bị quá lệch nhau

In [105]:
#Cầu thủ chưa ra sân nhưng lại có dữ liệu thi đấu (lỗi của data)
df.drop([723], inplace = True)


In [106]:
y = df['Position']
y.replace({'FW': 1, 'MF': 2, 'DF': 3}, inplace = True)
df.drop(columns='Position', inplace=True)

In [107]:
new_df = df.iloc[:,1:].apply(lambda x: round(x / df.Appearances, 2))
new_df.dtypes

Wins                     float64
Losses                   float64
Tackles                  float64
Blocked shots            float64
Interceptions            float64
Clearances               float64
Assists                  float64
Passes                   float64
Big chances created      float64
Crosses                  float64
Yellow cards             float64
Red cards                float64
Fouls                    float64
Offsides                 float64
Headed goals             float64
Goals with right foot    float64
Goals with left foot     float64
Hit woodwork             float64
dtype: object

In [108]:
#FILL NA BY MEAN
new_df.fillna(round(new_df.mean(), 2), inplace=True)
new_df
    

,Wins,Losses,Tackles,Blocked shots,Interceptions,Clearances,Assists,Passes,Big chances created,Crosses,Yellow cards,Red cards,Fouls,Offsides,Headed goals,Goals with right foot,Goals with left foot,Hit woodwork
0,0.39,0.40,1.18,0.23,0.70,1.28,0.06,28.00,0.09,1.15,0.11,0.0,0.71,0.11,0.01,0.04,0.03,0.02
1,0.21,0.44,1.12,0.07,1.23,5.60,0.00,55.44,0.00,0.07,0.02,0.0,0.51,0.05,0.02,0.00,0.00,0.00
2,0.39,0.40,1.18,0.23,0.70,1.28,0.06,28.00,0.09,1.15,0.11,0.0,0.71,0.11,0.01,0.04,0.03,0.02
3,0.39,0.40,1.18,0.23,0.70,1.28,0.06,28.00,0.09,1.15,0.11,0.0,0.71,0.11,0.01,0.04,0.03,0.02
4,0.00,1.00,1.00,0.00,1.00,1.00,0.00,49.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
903,0.47,0.27,0.78,0.67,0.45,0.22,0.12,26.88,0.29,3.47,0.12,0.0,0.86,0.18,0.00,0.00,0.12,0.04
904,0.63,0.25,1.11,0.63,0.24,0.32,0.13,41.38,0.21,2.00,0.10,0.0,0.63,0.06,0.02,0.03,0.17,0.03
905,0.27,0.42,1.69,0.28,1.94,1.21,0.05,33.68,0.03,0.92,0.18,0.0,1.04,0.03,0.00,0.01,0.01,0.01
906,0.43,0.35,1.67,0.18,1.37,0.51,0.10,51.49,0.10,2.02,0.10,0.0,0.86,0.00,0.00,0.00,0.00,0.02


## PHÂN CHIA TẬP TRAIN VÀ TẬP TEST

KÍCH THƯỚC TẬP TEST LÀ 20%, random state tùy ý

In [109]:
X_train, X_test, y_train, y_test = train_test_split(new_df, y, test_size=0.3, random_state=123)

### SGD Classifier

In [110]:
%%capture
param_grid = {'loss': ['hinge', 'huber', 'squared_hinge', 'perceptron'], 
            'penalty' : ['l1', 'l2', 'elasticnet'],
            'shuffle' :[True,False], 
            'fit_intercept': [True, False],
            'epsilon': [0.1, 0.5]}
            
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10)

sgd_grid = GridSearchCV( SGDClassifier(),param_grid = param_grid,
                          cv=cv, 
                          scoring='completeness_score', 
                          verbose=3) 
sgd_grid.fit(X_train,y_train)

In [111]:
print('Best Score: %s' % sgd_grid.best_score_)
print('Best Hyperparameters: %s' % sgd_grid.best_params_)

Best Score: 0.37564536530980897
Best Hyperparameters: {'epsilon': 0.5, 'fit_intercept': False, 'loss': 'huber', 'penalty': 'elasticnet', 'shuffle': True}


In [112]:
print(classification_report(sgd_grid.predict(X_test), y_test))

              precision    recall  f1-score   support

           1       0.02      0.20      0.04         5
           2       0.00      0.00      0.00         0
           3       0.98      0.40      0.57       268

    accuracy                           0.40       273
   macro avg       0.33      0.20      0.20       273
weighted avg       0.96      0.40      0.56       273



c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

### k-NN Classifier

In [113]:
%%capture
param_grid = {'n_neighbors': [5, 7, 10], 
            'algorithm' : ['ball_tree', 'kd_tree', 'brute'],
            'leaf_size': [30, 35, 40]}
            
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10)

knn_grid = GridSearchCV(KNeighborsClassifier(),param_grid = param_grid,
                          cv=cv, 
                          scoring='completeness_score', 
                          verbose=3) 
knn_grid.fit(X_train,y_train)

In [114]:
print('Best Score: %s' % knn_grid.best_score_)
print('Best Hyperparameters: %s' % knn_grid.best_params_)

Best Score: 0.17500831823347598
Best Hyperparameters: {'algorithm': 'ball_tree', 'leaf_size': 35, 'n_neighbors': 5}


In [115]:
print(classification_report(knn_grid.predict(X_test), y_test))

              precision    recall  f1-score   support

           1       0.60      0.53      0.56        53
           2       0.44      0.70      0.54        73
           3       0.83      0.62      0.71       147

    accuracy                           0.62       273
   macro avg       0.62      0.62      0.60       273
weighted avg       0.68      0.62      0.63       273



## Random Forest Classifier

In [116]:
%%capture
param_grid = {'n_estimators' : [100, 150, 200], 
            'criterion' : ["gini", "entropy"],
            'min_samples_split' : [2, 4, 6]}
            
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10)

rfc_grid = GridSearchCV(RandomForestClassifier(),param_grid = param_grid,
                          cv=cv, 
                          scoring='completeness_score', 
                          verbose=3) 
rfc_grid.fit(X_train,y_train)

In [117]:
print('Best Score: %s' % rfc_grid.best_score_)
print('Best Hyperparameters: %s' % rfc_grid.best_params_)

Best Score: 0.301164211398052
Best Hyperparameters: {'criterion': 'entropy', 'min_samples_split': 6, 'n_estimators': 200}


In [118]:
print(classification_report(rfc_grid.predict(X_test), y_test))

              precision    recall  f1-score   support

           1       0.57      0.71      0.64        38
           2       0.88      0.57      0.69       179
           3       0.47      0.93      0.63        56

    accuracy                           0.66       273
   macro avg       0.64      0.74      0.65       273
weighted avg       0.75      0.66      0.67       273



## AdaBoost Classifier

In [119]:
%%capture
param_grid = {'n_estimators' : [50, 100, 150], 
            'learning_rate' : [0.4, 0.2, 0.7, 1.0],
            'random_state' : [99, 388, 123]}
            
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10)

ab_grid = GridSearchCV(AdaBoostClassifier(),param_grid = param_grid,
                          cv=cv, 
                          scoring='completeness_score', 
                          verbose=3) 
ab_grid.fit(X_train,y_train)

In [120]:
print('Best Score: %s' % ab_grid.best_score_)
print('Best Hyperparameters: %s' % ab_grid.best_params_)

Best Score: 0.25393322641984406
Best Hyperparameters: {'learning_rate': 0.2, 'n_estimators': 50, 'random_state': 99}


In [121]:
print(classification_report(ab_grid.predict(X_test), y_test))

              precision    recall  f1-score   support

           1       0.45      0.64      0.52        33
           2       0.86      0.49      0.62       205
           3       0.28      0.89      0.43        35

    accuracy                           0.56       273
   macro avg       0.53      0.67      0.53       273
weighted avg       0.74      0.56      0.59       273



## Decision Tree Classifier

In [122]:
%%capture
param_grid = {'criterion': ['gini', 'entropy', 'log_loss'],
                            'splitter': ['best', 'random'],
                            'max_depth': [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]} 
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=1)

dt_grid = GridSearchCV(DecisionTreeClassifier(), 
                    param_grid, 
                    cv=cv, 
                    scoring='completeness_score', 
                    verbose=3)
                    
print(dt_grid.fit(X_train, y_train))

In [123]:
print('Best Score: %s' % dt_grid.best_score_)
print('Best Hyperparameters: %s' % dt_grid.best_params_)

Best Score: 0.25476249927326017
Best Hyperparameters: {'criterion': 'gini', 'max_depth': 5, 'splitter': 'best'}


In [124]:
print(classification_report(dt_grid.predict(X_test), y_test))

              precision    recall  f1-score   support

           1       0.64      0.55      0.59        55
           2       0.81      0.56      0.66       168
           3       0.41      0.90      0.56        50

    accuracy                           0.62       273
   macro avg       0.62      0.67      0.60       273
weighted avg       0.70      0.62      0.63       273



In [125]:
dt = classification_report(dt_grid.predict(X_test), y_test, output_dict=True)
ab = classification_report(ab_grid.predict(X_test), y_test, output_dict=True)
knn = classification_report(knn_grid.predict(X_test), y_test, output_dict=True)
rfc = classification_report(rfc_grid.predict(X_test), y_test, output_dict=True)
sgd = classification_report(sgd_grid.predict(X_test), y_test, output_dict=True)


c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

In [126]:
one = pd.DataFrame.from_dict([sgd['1'], knn['1'], rfc['1'], ab['1'], dt['1']])
two = pd.DataFrame.from_dict([sgd['2'], knn['2'], rfc['2'], ab['2'], dt['2']])
three = pd.DataFrame.from_dict([sgd['3'], knn['3'], rfc['3'], ab['3'], dt['3']])
acc =  pd.DataFrame.from_dict([sgd['accuracy'], knn['accuracy'], rfc['accuracy'], ab['accuracy'], dt['accuracy']])

one.set_axis(['SGD', 'k-NN', 'Random Forest', 'AdaBoost', 'Decision Tree'], inplace=True)
two.set_axis(['SGD', 'k-NN', 'Random Forest', 'AdaBoost', 'Decision Tree'], inplace=True)
three.set_axis(['SGD', 'k-NN', 'Random Forest', 'AdaBoost', 'Decision Tree'], inplace=True)
acc.set_axis(['SGD', 'k-NN', 'Random Forest', 'AdaBoost', 'Decision Tree'], inplace=True)
acc.set_axis(['Accuracy'], inplace=True, axis = 1)

In [127]:
one

,precision,recall,f1-score,support
SGD,0.021277,0.200000,0.038462,5
k-NN,0.595745,0.528302,0.560000,53
Random Forest,0.574468,0.710526,0.635294,38
AdaBoost,0.446809,0.636364,0.525000,33
Decision Tree,0.638298,0.545455,0.588235,55


In [128]:
two

,precision,recall,f1-score,support
SGD,0.000000,0.000000,0.000000,0
k-NN,0.439655,0.698630,0.539683,73
Random Forest,0.879310,0.569832,0.691525,179
AdaBoost,0.862069,0.487805,0.623053,205
Decision Tree,0.810345,0.559524,0.661972,168


In [129]:
three

,precision,recall,f1-score,support
SGD,0.981818,0.402985,0.571429,268
k-NN,0.827273,0.619048,0.708171,147
Random Forest,0.472727,0.928571,0.626506,56
AdaBoost,0.281818,0.885714,0.427586,35
Decision Tree,0.409091,0.900000,0.562500,50


In [130]:
acc

,Accuracy
SGD,0.399267
k-NN,0.622711
Random Forest,0.663004
AdaBoost,0.556777
Decision Tree,0.619048


Sau nhiều lần chạy thử, nhóm em đi đến những kết luận sau:
+ Mô hình SGD thường có độ chính xác khá thấp do nó thường dự đoán nghiêng hẳn về một nhãn (support > 90%) dẫn đến việc nhãn đó sẽ có precision cao vượt trội. Mô hình sẽ có ý nghĩa để nhận dạng liệu cầu thủ đó có chơi vị trí đang xét hay không
+ Nhóm mô hình k-Nearest Neightbor, AdaBoost và Desicion có hiệu suất tương đối giống nhau.
+ Mô hình Random Forest thường sẽ có hiệu suất tốt nhất với Accuracy cao và Recall ở cả ba nhẵn đều tương đối cao
+ Tuy nhiên nhìn chung, các mô hình đều có Accuracy không cao. Điều này là do thực tế của bộ dữ liệu. Ví dụ, có những cầu thủ có chỉ số liên quan đến phòng thủ tốt nhưng lại được phân công chơi ở các vị trí Tiền đạo hay Tiền vệ. Tuy nhiên việc mô hình có độ chính xác cũng nói lên một vấn đề thực tế là có tương đối nhiều cầu thủ đang không được khai thác hết tiềm nay của mình. 
+ Về mặt thời gian, AdaBoost là mô hình tốn nhiều thời gian để chạy nhất (6-7p), hơn 2 phút so với mô hình lâu nhì là Random Forest. Nhanh nhất trong các thuật toán trên là Desicion Tree với thời gian chạy không vượt quá 30 giây ở các lần chạy. Các thuật toán còn lại thì có tốc độ chạy tương đương nhau, thường là 2-3p cho mỗi lần chạy